# 6.2 Стекинг

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.base import clone

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from matplotlib import pyplot as plt

%matplotlib inline

In [2]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/covtype.data.gz', sep=',', header=None)[:10000]

In [3]:
df.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54
0,2596,51,3,258,0,510,221,232,148,6279,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2


In [4]:
df.shape

(10000, 55)

In [5]:
features = list(range(0, 54))
target = 54

df = df[(df[target] == 1) | (df[target] == 2)]

In [6]:
cover_train, cover_test = train_test_split(df, test_size=0.5)

cover_X_train, cover_y_train = cover_train[features], cover_train[target]
cover_X_test, cover_y_test = cover_test[features], cover_test[target]

In [7]:
scaler = StandardScaler()
cover_X_train = scaler.fit_transform(cover_X_train)
cover_X_test = scaler.transform(cover_X_test)

Stacking — еще один способ объединить несколько алгоритмов в один, который часто используется как в решении реальных задач из промышленной сферы, так и в конкурсах на платформах вроде Kaggle.  
Подход использует понятие _базовых классификаторов_, каждый из которых независимо обучается на некотором (возможно одном и том же) множестве признаков, а также _мета-классификатора_, использующего предсказания базовых классификаторов как признаки.

Для избежания переобучения будем разбивать обучающую выборку на фолды.  
Например, фолды при разбиении на три части:  
``==*``  
``=*=``  
``*==``  

Это требуется для того, чтобы получить новые признаки (ответы алгоритмов на первом уровне) на всей обучающей выборке, т.е. ответы алгоритма на тех объектах, которые не были использованы во время обучения. В примере выше мы будем использовать ответы алгоритма, полученные на объектах звездочках. _Важно_: на каждом фолде мы обучаем алгоритм заново.

In [8]:
def compute_meta_feature(clf, X_train, X_test, y_train, cv):
    """
    Computes meta-features using the classifier.
    
    :arg clf: scikit-learn classifier
    :args X_train, y_train: training set
    :arg X_test: testing set
    :arg cv: cross-validation folding
    """
    X_meta_train = np.zeros_like(y_train, dtype=np.float32)
    for train_fold_index, predict_fold_index in cv.split(X_train):
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]
        
        folded_clf = clone(clf)
        folded_clf.fit(X_fold_train, y_fold_train)
        X_meta_train[predict_fold_index] = folded_clf.predict_proba(X_fold_predict)[:, 1]
    
    meta_clf = clone(clf)
    meta_clf.fit(X_train, y_train)
    
    X_meta_test = meta_clf.predict_proba(X_test)[:, 1]
    
    return X_meta_train, X_meta_test

In [9]:
def generate_metafeatures(classifiers, X_train, X_test, y_train, cv):
    """
    Generates metafeatures using a list of classifiers.
    
    :arg classifiers: list of scikit-learn classifiers
    :args X_train, y_train: training set
    :arg X_test: testing set
    :arg cv: cross-validation folding
    """
    features = [
        compute_meta_feature(clf, X_train, X_test, y_train, cv)
        for clf in tqdm(classifiers)
    ]
    
    stacked_features_train = np.vstack([
        features_train for features_train, features_test in features
    ]).T

    stacked_features_test = np.vstack([
        features_test for features_train, features_test in features
    ]).T
    
    return stacked_features_train, stacked_features_test

In [10]:
np.random.seed(42)

In [11]:
# baseline
clf = GradientBoostingClassifier(n_estimators=300)
clf.fit(cover_X_train, cover_y_train)

accuracy_score(clf.predict(cover_X_test), cover_y_test)

0.7857646229739254

In [12]:
# stacking
cv = KFold(n_splits=10, shuffle=True)

stacked_features_train, stacked_features_test = generate_metafeatures([
    LogisticRegression(C=0.001, penalty='l1', solver='liblinear', max_iter=5000),
    LogisticRegression(C=0.001, penalty='l2', solver='liblinear', max_iter=5000),  
    RandomForestClassifier(n_estimators=300, n_jobs=-1),
    GradientBoostingClassifier(n_estimators=300)
], cover_X_train, cover_X_test, cover_y_train.values, cv)

100%|██████████| 4/4 [00:29<00:00,  7.37s/it]


In [13]:
# где это используется?
total_features_train = np.hstack([cover_X_train, stacked_features_train])
total_features_test = np.hstack([cover_X_test, stacked_features_test])

In [14]:
np.random.seed(42)
clf = LogisticRegression(penalty='none', solver='lbfgs')
clf.fit(stacked_features_train, cover_y_train)
accuracy_score(clf.predict(stacked_features_test), cover_y_test)

0.7906976744186046

In [15]:
# 6. Стекинг. Практика
import pandas as pd
import numpy as np

from sklearn.ensemble import (AdaBoostClassifier, GradientBoostingClassifier,
                              RandomForestClassifier, ExtraTreesClassifier)
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import clone
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.datasets import load_digits

from tqdm import tqdm

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats.distributions import randint

In [16]:
dataset = load_digits()
X, y = dataset['data'], dataset['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

In [ ]:
# Задание 6.1

# В скринкасте мы разобрали схему генерации признаков в стекинге, когда для тестовой выборки 
# алгоритм заново переобучался на всей тренировочной выборке. Реализуйте схему, когда вместо 
# этого производится агрегация ответов всех обученных на фолдах классификаторов на тестовой 
# выборке при помощи усреднения.

# Логика решения:
# 1) Создадим X_meta_test, заполним его нулями (по аналогии с X_meta_train);
# 2) Далее на каждом шаге, где мы обучаем folded_clf.fit(X_fold_train, y_fold_train) 
# и его предсказания на X_fold_predict запихиваем в X_meta_train[predict_fold_index] 
# добавим еще одну строку, где в X_meta_test будем добавлять предсказания вероятностей 
# folded_clf на X_test. Их можно сразу складывать друг с другом или сохранить много массивов, 
# тогда в конце их нужно будет все сложить, а потом делить на количество сплитов (количество 
# массивов равно количеству сплитов в кросс - валидации);
# 3) После цикла останется только усреднить все эти массивы, это и будет наш X_meta_test.

In [17]:
# За основу нужно взять следующий код:
def compute_meta_feature(clf, X_train, X_test, y_train, cv):
    """Эта функция подсчитывает признаки для мета-классификатора.    
     
    Они являются вероятностями классов при решении задачи многоклассовой классификации.    
    :arg clf: классификатор    
    :args X_train, y_train: обучающая выборка    
    :arg X_test: признаки тестовой выборки    
    :arg cv: класс, генерирующий фолды (KFold)    
    :returns X_meta_train, X_meta_test: новые признаки для обучающей и тестовой выборок
    """
    n_classes = len(np.unique(y_train))
    X_meta_train = np.zeros((len(X_train), n_classes), dtype=np.float32)
    for train_fold_index, predict_fold_index in cv.split(X_train):
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]

        folded_clf = clone(clf)
        folded_clf.fit(X_fold_train, y_fold_train)

        X_meta_train[predict_fold_index] = folded_clf.predict_proba(X_fold_predict)

    meta_clf = clone(clf)
    meta_clf.fit(X_train, y_train)

    X_meta_test = meta_clf.predict_proba(X_test)

    return X_meta_train, X_meta_test

In [18]:
def compute_meta_feature_mean(clf, X_train, X_test, y_train, cv):
    """
    Эта функция подсчитывает признаки для мета-классификатора. 
    Они являются вероятностями классов при решении задачи многоклассовой классификации.

    :arg clf: классификатор
    :args X_train, y_train: обучающая выборка
    :arg X_test: признаки тестовой выборки
    :arg cv: класс, генерирующий фолды (KFold)

    :returns X_meta_train, X_meta_test: новые признаки для обучающей и тестовой выборок
    """
    # ваш код

In [19]:
# Для следующих заданий используйте этот код:
def compute_meta_feature(clf, X_train, X_test, y_train, cv):
    
    n_classes = len(np.unique(y_train))
    X_meta_train = np.zeros((len(y_train), n_classes), dtype=np.float32)

    splits = cv.split(X_train)
    for train_fold_index, predict_fold_index in splits:
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]
        
        folded_clf = clone(clf)
        folded_clf.fit(X_fold_train, y_fold_train)
        
        X_meta_train[predict_fold_index] = folded_clf.predict_proba(X_fold_predict)
    
    meta_clf = clone(clf)
    meta_clf.fit(X_train, y_train)
    
    X_meta_test = meta_clf.predict_proba(X_test)
    
    return X_meta_train, X_meta_test


def generate_meta_features(classifiers, X_train, X_test, y_train, cv):
   
    features = [
        compute_meta_feature(clf, X_train, X_test, y_train, cv)
        for clf in tqdm(classifiers)
    ]
    
    stacked_features_train = np.hstack([
        features_train for features_train, features_test in features
    ])

    stacked_features_test = np.hstack([
        features_test for features_train, features_test in features
    ])
    
    return stacked_features_train, stacked_features_test

def compute_metric(clf, X_train=X_train, y_train=y_train, X_test=X_test):
    clf.fit(X_train, y_train)
    y_test_pred = clf.predict(X_test)
    # у_test в аргументы бы добавить
    return np.round(f1_score(y_test, y_test_pred, average='macro'), 6)

In [20]:
# Во всех случаях, когда модель принимает random_state — обязательно указывайте его равным 42
# На разных версиях sklearn ответы могут отличаться
# В задании может понадобиться, а может не понадобиться нормализация и это нужно проверить 
# во время решения задания.

In [ ]:
# Задание 6.2

# Используйте функцию generate_meta_features для стекинга следующих алгоритмов:
#  - логистическая регрессия с L1-регуляризацией, C=0.001, солвер — 'saga', схема работы 
#  - мультиклассовой классификации — one-vs-rest, максимальное допустимое количество итераций — 2000
#  - логистическая регрессия с L2-регуляризацией, C=0.001, солвер — 'saga', схема работы 
#  - мультиклассовой классификации — multinomial, максимальное допустимое количество итераций — 2000
#  - случайный лес из 300 деревьев
#  - градиентный бустинг из 200 деревьев
# Как мета-алгоритм используйте логистическую регрессию без регуляризации со схемой работы 
# мультиклассовой классификации — auto и солвером 'lbfgs'.
# Посчитайте качество при помощи передачи новых признаков в функцию compute_metric.

In [22]:
cv = KFold(n_splits=10, shuffle=True, random_state=42)

# новые фичи без таргетов, разбитые на две выборки
stacked_features_train, stacked_features_test = generate_meta_features([
    LogisticRegression(
        C=0.001, penalty='l1', solver='saga', multi_class='ovr', max_iter=2000
        ),
    LogisticRegression(
        C=0.001, penalty='l2', solver='saga', multi_class='multinomial', max_iter=2000
        ),  
    RandomForestClassifier(n_estimators=300, n_jobs=-1),
    GradientBoostingClassifier(n_estimators=200)
], X_train, X_test, y_train, cv)

100%|██████████| 4/4 [03:38<00:00, 54.52s/it]


In [23]:
np.random.seed(42)
clf = LogisticRegression(penalty='none', solver='lbfgs', multi_class='auto')

compute_metric(clf, stacked_features_train, y_train, stacked_features_test)

0.983805

In [ ]:
# Задание 6.3

# Используйте функцию generate_meta_features для стекинга следующих алгоритмов:
#  - случайный лес из 300 деревьев
#  - случайный лес из 200 экстремальных деревьев
# Как мета-алгоритм используйте логистическую регрессию без регуляризации со схемой работы 
# мультиклассовой классификации — auto и солвером 'lbfgs'.
# Посчитайте качество при помощи передачи новых признаков в функцию compute_metric.

In [29]:
stacked_features_train, stacked_features_test = generate_meta_features([
    RandomForestClassifier(n_estimators=300, random_state=42, n_jobs=-1),
    ExtraTreesClassifier(n_estimators=200, random_state=42, n_jobs=-1),
], X_train, X_test, y_train, cv)

100%|██████████| 2/2 [00:23<00:00, 11.61s/it]


In [30]:
np.random.seed(42)
clf = LogisticRegression(penalty='none', solver='lbfgs', multi_class='auto')

compute_metric(clf, stacked_features_train, y_train, stacked_features_test)

0.982421

In [ ]:
# Задание 6.4

# Используйте функцию generate_meta_features для стекинга следующих алгоритмов:
#  - метод ближайшего соседа (k-NN) со стандартными параметрами
#  - случайный лес из 300 экстремальных деревьев
# Как мета-алгоритм используйте логистическую регрессию без регуляризации со схемой работы 
# мультиклассовой классификации — auto и солвером 'lbfgs'.
# Посчитайте качество при помощи передачи новых признаков в функцию compute_metric.

In [31]:
stacked_features_train, stacked_features_test = generate_meta_features([
    KNeighborsClassifier(n_jobs=-1),
    ExtraTreesClassifier(n_estimators=300, random_state=42, n_jobs=-1),
], X_train, X_test, y_train, cv)

100%|██████████| 2/2 [00:11<00:00,  5.82s/it]


In [32]:
np.random.seed(42)
clf = LogisticRegression(penalty='none', solver='lbfgs', multi_class='auto')

compute_metric(clf, stacked_features_train, y_train, stacked_features_test)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


0.989904

In [ ]:
# Задание 6.5

# Используйте функцию generate_meta_features для стекинга следующих алгоритмов:
#  - логистическая регрессия с L1-регуляризацией, C=0.001, солвер — 'saga', схема работы 
#    мультиклассовой классификации — one-vs-rest, максимальное допустимоей количество итераций — 2000
#  - метод ближайшего соседа со стандартными параметрами
#  - случайный лес из 300 экстремальных деревьев
#  - AdaBoost со стандартными параметрами
# Как мета-алгоритм используйте логистическую регрессию без регуляризации со схемой работы 
# мультиклассовой классификации — auto и солвером 'lbfgs'.
# Посчитайте качество при помощи передачи новых признаков в функцию compute_metric.

In [33]:
stacked_features_train, stacked_features_test = generate_meta_features([
    LogisticRegression(
        C=0.001, penalty='l1', solver='saga', multi_class='ovr', max_iter=2000
        ),
    KNeighborsClassifier(n_jobs=-1),
    ExtraTreesClassifier(n_estimators=300, random_state=42, n_jobs=-1),
    AdaBoostClassifier(random_state=42),
], X_train, X_test, y_train, cv)

100%|██████████| 4/4 [01:00<00:00, 15.04s/it]


In [34]:
np.random.seed(42)
clf = LogisticRegression(penalty='none', solver='lbfgs', multi_class='auto')

compute_metric(clf, stacked_features_train, y_train, stacked_features_test)

0.987404

In [38]:
def compute_meta_feature(clf, X_train, X_test, y_train, cv):
    
    n_classes = len(np.unique(y_train))
    X_meta_train = np.zeros((len(y_train), n_classes), dtype=np.float32)

    splits = cv.split(X_train, y_train)
    for train_fold_index, predict_fold_index in splits:
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]
        
        folded_clf = clone(clf)
        folded_clf.fit(X_fold_train, y_fold_train)
        
        X_meta_train[predict_fold_index] = folded_clf.predict_proba(X_fold_predict)
    
    meta_clf = clone(clf)
    meta_clf.fit(X_train, y_train)
    
    X_meta_test = meta_clf.predict_proba(X_test)
    
    return X_meta_train, X_meta_test

In [35]:
# Задание 6.6

# Используйте функцию generate_meta_features для стекинга следующих алгоритмов:
#  - случайный лес из 300 деревьев
#  - случайный лес из 300 экстремальных деревьев

# Для генерации фолдов используйте класс StratifiedKFold, который позволяет делать так 
# называемые стратифицированные разбиения (в каждом фолде будет одинаковое соотношение классов).
# Для корректной работы необходимо подправить код в функции compute_meta_feature. 

# Как мета-алгоритм используйте логистическую регрессию без регуляризации со схемой работы 
# мультиклассовой классификации — auto и солвером 'lbfgs'.

# Посчитайте качество при помощи передачи новых признаков в функцию compute_metric. 
# Количество фолдов = 10

In [39]:
skf = StratifiedKFold(n_splits=10, shuffle=False, random_state=None)

stacked_features_train, stacked_features_test = generate_meta_features([
    RandomForestClassifier(n_estimators=300, random_state=42, n_jobs=-1),
    ExtraTreesClassifier(n_estimators=300, random_state=42, n_jobs=-1),
], X_train, X_test, y_train, skf)


100%|██████████| 2/2 [00:23<00:00, 11.72s/it]


In [40]:
np.random.seed(42)
clf = LogisticRegression(penalty='none', solver='lbfgs', multi_class='auto')

compute_metric(clf, stacked_features_train, y_train, stacked_features_test)

0.979448

In [ ]:
# Задание 6.7

# В предыдущей задаче измените 10 фолдов на 20. Укажите полученное качество.

In [41]:
skf = StratifiedKFold(n_splits=20, shuffle=False, random_state=None)

stacked_features_train, stacked_features_test = generate_meta_features([
    RandomForestClassifier(n_estimators=300, random_state=42, n_jobs=-1),
    ExtraTreesClassifier(n_estimators=300, random_state=42, n_jobs=-1),
], X_train, X_test, y_train, skf)


100%|██████████| 2/2 [00:48<00:00, 24.37s/it]


In [42]:
np.random.seed(42)
clf = LogisticRegression(penalty='none', solver='lbfgs', multi_class='auto')

compute_metric(clf, stacked_features_train, y_train, stacked_features_test)

0.983963

In [ ]:
# Задание 6.8

# В предыдущей задаче укажите количество фолдов равным 5 и поменяйте мета-алгоритм 
# на случайный лес со стандартными параметрами. Укажите полученное качество.

In [43]:
skf = StratifiedKFold(n_splits=5, shuffle=False, random_state=None)

stacked_features_train, stacked_features_test = generate_meta_features([
    RandomForestClassifier(n_estimators=300, random_state=42, n_jobs=-1),
    ExtraTreesClassifier(n_estimators=300, random_state=42, n_jobs=-1),
], X_train, X_test, y_train, skf)

np.random.seed(42)
clf = RandomForestClassifier(random_state=42, n_jobs=-1)

compute_metric(clf, stacked_features_train, y_train, stacked_features_test)


100%|██████████| 2/2 [00:14<00:00,  7.21s/it]


0.985118

In [44]:
# Задание 6.9

# В предыдущей задаче поменяйте мета-алгоритм на метод ближайших соседей (k-NN) со 
# стандартными параметрами. Укажите полученное качество.

In [45]:
skf = StratifiedKFold(n_splits=5, shuffle=False, random_state=None)

stacked_features_train, stacked_features_test = generate_meta_features([
    RandomForestClassifier(n_estimators=300, random_state=42, n_jobs=-1),
    ExtraTreesClassifier(n_estimators=300, random_state=42, n_jobs=-1),
], X_train, X_test, y_train, skf)

np.random.seed(42)
clf = KNeighborsClassifier(n_jobs=-1)

compute_metric(clf, stacked_features_train, y_train, stacked_features_test)


100%|██████████| 2/2 [00:14<00:00,  7.25s/it]


0.984162

In [ ]:
# Задание 6.10

# В предыдущей задаче поменяйте мета-алгоритм на градиентный бустинг со стандартными параметрами. 
# Укажите полученное качество.

In [46]:
skf = StratifiedKFold(n_splits=5, shuffle=False, random_state=None)

stacked_features_train, stacked_features_test = generate_meta_features([
    RandomForestClassifier(n_estimators=300, random_state=42, n_jobs=-1),
    ExtraTreesClassifier(n_estimators=300, random_state=42, n_jobs=-1),
], X_train, X_test, y_train, skf)

np.random.seed(42)
clf = GradientBoostingClassifier()

compute_metric(clf, stacked_features_train, y_train, stacked_features_test)


100%|██████████| 2/2 [00:14<00:00,  7.19s/it]


0.984925

In [ ]:
# Задание 6.11

# Используйте функцию generate_meta_features для стекинга следующих алгоритмов:
#  - случайный лес из 300 деревьев, критерий Джини, максимальная глубина — 24
#  - случайный лес из 300 экстремальных деревьев
# Для генерации фолдов используйте класс StratifiedKFold, который позволяет делать так 
# называемые стратифицированные разбиения (в каждом фолде будет одинаковое соотношение классов). 
# StratifiedKFold строго только один раз перетасовывает данные и разбивает их на несколько частей. 
# Каждую такую часть он сможет использовать как тестовый набор.

# Для генерации фолдов используйте класс StratifiedKFold и поправленный вами ранее код 
# в функции compute_meta_feature. Выполните разбиение на 3 фолда.

# Как мета-алгортм используйте случайный лес из 100 экстремальных деревьев. 
# Посчитайте качество при помощи передачи новых признаков в функцию compute_metric.

In [47]:
skf = StratifiedKFold(n_splits=3, shuffle=False, random_state=None)

stacked_features_train, stacked_features_test = generate_meta_features([
    RandomForestClassifier(
        n_estimators=300,
        criterion='gini',
        max_depth=24,
        random_state=42,
        n_jobs=-1,
        ),
    ExtraTreesClassifier(n_estimators=300, random_state=42, n_jobs=-1),
], X_train, X_test, y_train, skf)

np.random.seed(42)
clf = ExtraTreesClassifier(n_estimators=100, random_state=42, n_jobs=-1)

compute_metric(clf, stacked_features_train, y_train, stacked_features_test)


100%|██████████| 2/2 [00:10<00:00,  5.21s/it]


0.984162

In [ ]:
# Задание 6.12

# Обучите на тренировочной выборке следующие алгоритмы:
#  - случайный лес из 300 деревьев, критерий Джини, максимальная глубина — 24
#  - случайный лес из 300 экстремальных деревьев
#  - логистическую регрессию со стандартными параметрами
# Усредните их ответы на тестовой выборке и посчитайте качество аналогично функции 
# compute_metric (F1-score с макро-усреднением, округленный до 6 знака).

# train
# predict on test
# average
# F1

In [48]:
rf = RandomForestClassifier(
    n_estimators=300,
    criterion='gini',
    max_depth=24,
    random_state=42,
    n_jobs=-1
    ).fit(X_train, y_train)

rd_y_pred = rf.predict(X_test)

In [50]:
extra_rf = ExtraTreesClassifier(
    n_estimators=300,
    random_state=42,
    n_jobs=-1
    ).fit(X_train, y_train)
extra_rd_y_pred = extra_rf.predict(X_test)

In [51]:
lg = LogisticRegression(random_state=42, n_jobs=-1).fit(X_train, y_train)
lg_y_pred = lg.predict(X_test)

In [52]:
lg_y_pred.shape

(360,)

In [55]:
ave_pred = np.array([rd_y_pred, extra_rd_y_pred, lg_y_pred]).mean(axis=0)
ave_pred.shape

(360,)

In [60]:
np.round(ave_pred).astype('int')

array([6, 9, 3, 7, 2, 1, 5, 2, 5, 2, 1, 9, 4, 0, 4, 2, 3, 7, 8, 8, 4, 3,
       9, 7, 5, 6, 3, 5, 6, 3, 4, 9, 1, 4, 4, 6, 9, 4, 7, 6, 6, 9, 1, 3,
       6, 1, 3, 0, 6, 5, 5, 1, 7, 5, 6, 0, 9, 0, 0, 1, 0, 4, 5, 2, 4, 5,
       7, 0, 7, 5, 9, 5, 5, 4, 7, 0, 4, 5, 5, 9, 9, 0, 2, 3, 8, 0, 6, 4,
       4, 9, 1, 2, 8, 3, 5, 2, 9, 0, 4, 4, 4, 3, 5, 3, 1, 3, 5, 9, 4, 2,
       7, 7, 4, 4, 1, 9, 2, 7, 8, 7, 2, 6, 9, 4, 0, 7, 2, 7, 5, 8, 7, 5,
       7, 8, 0, 6, 6, 4, 2, 8, 0, 9, 4, 6, 9, 9, 6, 9, 0, 5, 5, 6, 6, 0,
       6, 4, 3, 9, 3, 7, 7, 2, 9, 0, 5, 5, 5, 6, 5, 9, 9, 8, 4, 2, 1, 3,
       7, 7, 2, 2, 3, 9, 8, 0, 3, 2, 2, 5, 6, 9, 9, 4, 1, 4, 4, 2, 3, 6,
       4, 8, 5, 9, 5, 7, 3, 9, 4, 8, 1, 5, 4, 4, 9, 6, 1, 8, 6, 0, 4, 5,
       2, 7, 3, 6, 4, 5, 6, 0, 3, 2, 3, 6, 7, 1, 6, 1, 4, 7, 6, 7, 6, 5,
       5, 1, 5, 2, 8, 8, 9, 6, 7, 6, 2, 2, 2, 3, 4, 8, 8, 3, 6, 0, 9, 7,
       7, 0, 1, 0, 4, 5, 1, 5, 3, 6, 0, 4, 1, 0, 0, 3, 6, 5, 9, 7, 3, 5,
       5, 9, 9, 8, 5, 3, 3, 2, 0, 5, 8, 3, 4, 0, 2,

In [61]:
round(f1_score(y_test, np.round(ave_pred).astype('int'), average='macro'), 6)
# не принимается ответ, попробую стандартизировать данные

0.960096

In [63]:
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)

In [65]:
rf = RandomForestClassifier(
    n_estimators=300,
    criterion='gini',
    max_depth=24,
    random_state=42,
    n_jobs=-1
    ).fit(X_train_std, y_train)

rd_y_pred = rf.predict(X_test_std)

extra_rf = ExtraTreesClassifier(
    n_estimators=300,
    random_state=42,
    n_jobs=-1
    ).fit(X_train_std, y_train)
extra_rd_y_pred = extra_rf.predict(X_test_std)

lg = LogisticRegression(random_state=42, n_jobs=-1).fit(X_train_std, y_train)
lg_y_pred = lg.predict(X_test_std)

ave_pred = np.array([rd_y_pred, extra_rd_y_pred, lg_y_pred]).mean(axis=0)
ave_pred.shape

(360,)

In [66]:
round(f1_score(y_test, np.round(ave_pred).astype('int'), average='macro'), 6)
# не помогло, все равно платформа не приняла ответ

0.959129